In [17]:
# !ssh-keygen -t rsa -b 4096 -C "fabianbernard49@gmail.com" -f ~/.ssh/id_rsa -N "" # create ssh pair key
# !cat ~/.ssh/id_rsa.pub # print it out
!ssh-keyscan -H github.com >> ~/.ssh/known_hosts # make github a knownhost
!ssh -T git@github.com #aunthenticate github
!git config --global user.name "coded-by-49" # make yourself a known host
!git config --global user.email "fabianbernard49@gmail.com"
%cd /content/drive/MyDrive/Git_repos
!git clone git@github.com:coded-by-49/Telecom-churn-model.git

# github.com:22 SSH-2.0-da7dc22f
# github.com:22 SSH-2.0-da7dc22f
# github.com:22 SSH-2.0-da7dc22f
# github.com:22 SSH-2.0-da7dc22f
# github.com:22 SSH-2.0-da7dc22f
Hi coded-by-49! You've successfully authenticated, but GitHub does not provide shell access.
/content/drive/MyDrive/Git_repos
Cloning into 'Telecom-churn-model'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 4), reused 8 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 10.74 KiB | 1.34 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [18]:
from itertools import groupby
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from IPython.display import clear_output
import tensorflow as tf
fc = tf.compat.v2.feature_column
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# GET OUR THE FILE READY
from google.colab import drive
import urllib
drive.mount('/content/drive')
tele_file = "/content/drive/MyDrive/GOOGLE COLLAB PROJECTS/WA_Fn-UseC_-Telco-Customer-Churn.csv"
df_tele = pd.read_csv(tele_file)

# LEARNIN ALGORITHM
from sklearn.linear_model import LogisticRegression
df_tele.drop("customerID", axis = 1, inplace = True) # removed this unimportant feature

# CLEANING OF DATA
df_tele["TotalCharges"] = df_tele["TotalCharges"].replace(" ", np.nan) # replacing empty strings np.nan, to that i can convert it to float
df_tele.dropna(subset = ["TotalCharges"], inplace=True) # cleaded out the array by dropping does empty values within the Totalcharges feature
df_tele["TotalCharges"] = pd.to_numeric(df_tele["TotalCharges"]) # coverted to float because of the kind of content within totalcharges

# ENCODING OF ALL CATEGORICAL FEATURES
encoded = pd.get_dummies(df_tele, drop_first= True ) # this is one-hot key encoding (we want to transform all our categorical columns into )
encoded.head()

# CHANGING THE DATATYPE OF BINARY CATEGORICAL FEATURES
bool_cols = encoded.select_dtypes(include = "bool").columns #select all columns with bool datatype
encoded[bool_cols] = encoded[bool_cols].astype(int) #change all of them to int (0 and 1)

encoded["tenure"] = encoded["tenure"].astype(float)
all_numerical_features = encoded.select_dtypes(include = np.number).columns.tolist()

needed_numerical_feature = [feature for feature in all_numerical_features if encoded[feature].nunique()>2] # getting the numerical feature titles ready

Labels = encoded["Churn_Yes"]
Features = encoded.drop("Churn_Yes", axis = 1)

# OBTANING OUR TRANING AND TEST DATA
X_train, X_test, Y_train, Y_test = train_test_split(Features,Labels, test_size = 0.2, random_state = 42 )
# SCALLING AND PREPARING OUR TRAINING AND TESTING DATA
Scaler = StandardScaler()

# let us fit and tranform  our training numerical-features
X_train[needed_numerical_feature] = Scaler.fit_transform(X_train[needed_numerical_feature])

# let us apply the scaling parameters gotten from above and fit it to our testing numerical-features
X_test[needed_numerical_feature] = Scaler.transform(X_test[needed_numerical_feature])

log_regression = LogisticRegression(max_iter=1000) # is max_iter really needed?
log_regression.fit(X_train, Y_train) # FIT THIS WHERE OUR WEIGHT AND BIAS ARE DERVIED FROM

# print(f"Testing accuracy: {log_regression.score(X_test, Y_test)}") # SCORE THE TEST DATA

# LOGISTC REGRESSION FROM SCRATCH

class Logistic_regression:
  def __init__(self, learning_rate = 0.01, n_iters = 1000):
    self.learning_rate = learning_rate
    self.n_iters = n_iters

  def sigmoid(self,z):
    return 1/(1+np.exp(-z))

  '''
  CORE
  This is responsible deriving weight and bias of a several feature(s).
  This is important, because when we want to derive predictions on any future dataset this are the exact values we will use
  '''

  def fit(self,X,Y):
    self.num_samples, self.num_features = X.shape #lets count the number of features and dataitems(rows)
    self.weights = np.zeros(self.num_features)
    self.bias = 0

    for _ in range(self.n_iters):
      Y_num_prediction = np.dot(X, self.weights) + self.bias
      Y_predicted = self.sigmoid(Y_num_prediction)

      # implementing gradient descent

      # we are implementing the average our two derivatives of the loss function
      dw = np.dot(X.T,(Y_predicted - Y))/self.num_samples
      db = np.mean((Y_predicted - Y))

      # adjust weights after each iteration with learning rate
      self.weights = self.weights - self.learning_rate * dw
      self.bias = self.bias - self.learning_rate * db

      # stoping condition with eculidean normalisation
      if np.linalg.norm(db) < 0.001 and np.linalg.norm(dw) < 0.001:
        break

# This function is used for determining the accuracy score
  def score(self, X_test,Y_test):
    Y_num_prediction = np.dot(X_test, self.weights) + self.bias
    self.Y_predicted = self.sigmoid(Y_num_prediction)
    self.Y_predicted = np.where(self.Y_predicted > 0.5, 1, 0)
    accuracy =  np.mean(self.Y_predicted == Y_test)
    return accuracy

  def Confusion_matrix(self,predicted_class,actual_class):
    confusion_matrix = {
    'TP': 0,
    'TN': 0,
    'FP': 0,
    'FN': 0
    }
    for i in range(len(actual_class)):
      if predicted_class[i] == actual_class.iloc[i]:
        if predicted_class[i] == 1:
            confusion_matrix["TP"] += 1
        else:
            confusion_matrix["TN"] += 1
      else:
        if actual_class.iloc[i] == 0:
            confusion_matrix["FP"] += 1
        else:
            confusion_matrix["FN"] += 1
    return confusion_matrix

test_model = Logistic_regression()
test_model.fit(X_train, Y_train)
print(f"Accuracy on test models: {test_model.score(X_test,Y_test)}")
"TESTING MY CONFUSION MATRIX"
predicted_class = test_model.Y_predicted
actual_class = Y_test
print(f"Confusion matrix = {test_model.Confusion_matrix(predicted_class,actual_class)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Accuracy on test models: 0.7903340440653873
Confusion matrix = {'TP': 171, 'TN': 941, 'FP': 92, 'FN': 203}


In [25]:
# %cd /content/drive/MyDrive/Git_repos/Telecom-churn-model/

!cp "/content/drive/MyDrive/Colab Notebooks/Tele_churn.ipynb" /content/drive/MyDrive/Git_repos/Telecom-churn-model/
!git status
!git add Tele_churn.ipynb
!git commit -m "Proper update to git hu"

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Tele_churn.ipynb

nothing added to commit but untracked files present (use "git add" to track)
